# 新闻问答 v0.3

初步结论：

- 回答不稳定，相同的模型，相同的问题，有时候正确，有时候就查不到。
- 有些模型好一点，有些模型问题比较大
- 可能改进的方向
    - 使用中文提示词？

## 通用设置

In [1]:
%%time

INDEX_PATH="news-index"

# TODO: 改为使用环境变量

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.34 µs


## 初始化

### 清除文档存储

In [2]:
%%time
!rm -rf $INDEX_PATH 
# 怎么拿到python变量的？

CPU times: user 3.97 ms, sys: 8.12 ms, total: 12.1 ms
Wall time: 114 ms


### 加载模型和嵌入

In [3]:
%%time

# 加载llm和embeddings
%run ../utils2.py

from llama_index.core import Settings

# Settings.llm=get_llm("gpt-3.5-turbo")
# Settings.llm=get_llm("glm-4-flash")
# Settings.llm=get_llm('qwen-turbo')
Settings.llm=get_llm('ERNIE-4.0-8K') # 目前表现最好的
# Settings.llm=get_llm()
Settings.embed_model=get_embedding()

CPU times: user 3.33 s, sys: 371 ms, total: 3.7 s
Wall time: 3.39 s


### 加载新闻问答库

In [4]:
%%time

%run ./news_v03.py

CPU times: user 9.63 ms, sys: 4.18 ms, total: 13.8 ms
Wall time: 13.1 ms


### 创建 Agent

In [5]:
%%time

from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools([
    get_news_summary_tool,
    get_query_engine_tool()], verbose=True)

Parsing nodes: 0it [00:00, ?it/s]

Summarizing documents: 0it [00:00, ?it/s]

Generating embeddings: 0it [00:00, ?it/s]

CPU times: user 436 ms, sys: 16.3 ms, total: 452 ms
Wall time: 449 ms


## 使用

### 第一个新闻 - 新闻的摘要

In [6]:
%%time

response = agent.stream_chat("https://www.guancha.cn/industry-science/2024_06_24_739086.shtml",chat_history=[])
response.print_response_stream()


{"url": "https://www.guancha.cn/industry-science/2024_06_24_739086.shtml"}
```
Thought: The user has provided a URL, indicating that they want information from a specific news article. To retrieve the content of the news article, I will use the get_news_summary tool.
Action: get_news_summary
Action Input: {"url": "https://www.guancha.cn/industry-science/2024_06_24_739086.shtml"}
```json
{"url": "https://www.guancha.cn/industry-science/2024_06_24_739086.shtml"}
```CPU times: user 11.4 s, sys: 2.57 s, total: 13.9 s
Wall time: 17.5 s


### 对第一个新闻提问

In [7]:
%%time

response = agent.stream_chat("星际客机是否能将宇航员带回地球？",chat_history=[])
response.print_response_stream()

星际客机”是否能将宇航员带回地球的问题，以下是我的回答：

**“星际客机”目前面临技术难题，尚未成功将宇航员带回地球**。具体来说，虽然“星际客机”已成功地将宇航员送往国际空间站，但在返回地球的过程中遇到了一系列技术问题，导致宇航员的返回计划受阻。

1. **技术故障**：据报道，“星际客机”在太空停留期间遭遇了多次氦气泄漏和推进器故障。这些问题严重影响了飞船的姿态控制系统和其他关键系统，增加了任务的风险系数，也使得返回地球的计划变得复杂和不确定。

2. **返回日期推迟**：由于上述技术问题，美国航天局（NASA）已经多次推迟了“星际客机”的返回日期。最初计划于6月14日返回，但随后因需更多时间进行技术审查和修复工作而推迟。目前，具体的返回日期仍未确定，但预计可能在不早于7月2日。

3. **备选方案**：在面临返回困难的情况下，NASA表示已有备选方案，即使用SpaceX的“龙飞船”将宇航员带回地球。这表明“星际客机”目前尚未能独立完成将宇航员安全返回地球的任务。

综上所述，“星际客机”在将宇航员带回地球方面仍面临重大挑战。虽然波音公司和NASA正在积极努力解决问题并确保宇航员的安全返回，但目前的情况仍充满不确定性。因此，对于“星际客机”是否能将宇航员带回地球的问题，目前的答案是否定的。CPU times: user 22.4 s, sys: 4.81 s, total: 27.2 s
Wall time: 33.8 s


### 对第一个新闻的第二个问题

In [8]:
%%time

response = agent.stream_chat("星际客机项目支出是多少，是否超出预算？",chat_history=[])
response.print_response_stream()

客机项目的支出和预算情况，可以归纳为以下几点：

1. **项目支出**：
   - 波音公司在“星际客机”项目上的投入是巨大的。根据波音公司公布的财报信息，该项目已经累计亏损超过11.4亿美元（约合81.62亿元人民币）。这一数字显示了波音公司在该项目上的巨大投入。

2. **预算超支情况**：
   - “星际客机”项目确实已经超出了其原始预算。根据报道，该项目不仅已较原计划严重滞后，而且超出了预算达15亿美元之多。这表明项目的实际支出已经远远超过了最初的预算安排。

综上所述，“星际客机”项目的支出巨大，并且已经超出了其原始预算。这反映了该项目在研发和执行过程中面临的挑战和困难。CPU times: user 9.38 s, sys: 1.94 s, total: 11.3 s
Wall time: 19.4 s


### 第二个新闻

In [9]:
%%time

response = agent.stream_chat("https://www.guancha.cn/internation/2024_06_26_739350.shtml",chat_history=[])
response.print_response_stream()


{"url": "https://www.guancha.cn/internation/2024_06_26_739350.shtml"}
```
Thought: The user has provided a URL, which suggests that they want information from a specific news article. Since the question is in the form of a URL, I will use the get_news_summary tool to retrieve a summary of the news article.
Action: get_news_summary
Action Input: {"url": "https://www.guancha.cn/internation/2024_06_26_739350.shtml"}

Observation: 需要用户提供，即新闻摘要内容。

Thought: Since the observation requests the news summary content from the user, I will assume that the user will provide the summary in the next message. Once I have the summary, I can answer any questions related to the content of the news article. However, since there is no specific question asked yet, I will wait for further instructions from the user.

Answer: Please provide the news summary or ask a specific question about the content of the article.CPU times: user 18.2 s, sys: 3.86 s, total: 22.1 s
Wall time: 25 s


### 对第二个问题提问

In [10]:
%%time

response = agent.stream_chat("谁被任命为新的北约秘书长？",chat_history=[])
response.print_response_stream()

首相马克·吕特被任命为新的北约秘书长**。以下是关于这一任命的详细信息：

1. 任命时间：2024年6月26日，北约成员国特使会议确认了这一任命。

2. 接任时间：吕特将于2024年10月2日正式接替现任北约秘书长斯托尔滕贝格，任期五年。

3. 支持情况：吕特在任命前已经获得了包括美、英、法、德等多个国家的支持。此外，他原先的唯一竞争对手、罗马尼亚总统克劳斯·约翰尼斯已宣布退出竞选，使得吕特的任命几无悬念。

4. 确认过程：北约秘书长提名需要全体32个成员国一致同意。成员国的大使预计在北约的主要决策机构——北大西洋理事会的会议上正式确认了吕特为新任秘书长。CPU times: user 14.4 s, sys: 3.23 s, total: 17.6 s
Wall time: 22.2 s
